# ch 12_4. word cloud

이전 챕터에서 2022년 한국 야구 데이터 셋을 토큰화 했습니다. 이번 챕터에서는 가장 기본적인 자연어 데이터 시각화 기법인 워드 클라우드에 대해서 알아보겠습니다. 워드 클라우드는 특정 단어와 함께 등장한 단어의 빈도 수를 집계해서, 어떤 단어가 연관되어 있는지를 시각적으로 표현하는 기법입니다.

In [1]:
!pip install wordcloud

                                              0.0/151.4 kB ? eta -:--:--
     -------------------------------------- 151.4/151.4 kB 9.4 MB/s eta 0:00:00


## 선수 이름 데이터 준비
우리가 만들어보고 싶은 건 특정 선수 이름이 주어지면, 그 선수와 함께 언급되는 단어들을 시각적으로 보여주는 것입니다. 이를 위해서 먼저 토큰화 한 뉴스 기사 데이터에서 특정 선수가 언급되면, 함께 언급된 단어들의 빈도수를 세어 딕셔너리 형태로 만들어보겠습니다. 

먼저 기사에 선수 이름이 포함되었는지 여부를 판단하기 위해 크롤러 시간에 짰던 코드를 재사용합니다.

In [110]:
import pandas as pd

In [111]:
df = pd.read_csv("./data/baseball_players.csv", index_col=["구단"])

In [112]:
df.head()

,투수,포수,내야수,외야수
구단,,,,
롯데,김진욱\n한현희\n최준용\n박세웅\n구승민\n김상수\n반즈\n신정락\n최이준\n김원...,유강남\n정보근,안치홍\n한동희\n김민수\n노진혁\n박승욱\n이학주\n전준우\n정훈,안권수\n렉스\n김민석\n고승민\n윤동희
SSG,고효준\n서진용\n김광현\n맥카티\n노경은\n박민호\n문승원\n오원석\n임준섭\n신...,김민식\n조형우,최정\n박성한\n김건웅\n최주환\n전의산\n최경모\n김성현,추신수\n에레디아\n한유섬\n오태곤\n김정민\n최상민
LG,임찬규\n함덕주\n정우영\n진해수\n켈리\n이정용\n배재준\n박명근\n김진성\n플럿...,김기연\n박동원,오지환\n서건창\n김민성\n문보경\n송찬의\n정주현,박해민\n김현수\n오스틴\n신민재\n홍창기\n문성주
두산,박치국\n홍건희\n이병헌\n김동주\n최지강\n알칸타라\n김명신\n곽빈\n최원준\n김...,장승현\n양의지,허경민\n강승호\n전민재\n김재호\n양석환\n이유찬,로하스\n정수빈\n김재환\n강진성\n조수행\n양찬열\n송승환
NC,송명기\n페디\n김태현\n김영규\n김시훈\n임정호\n하준영\n이용준\n신민혁\n김진...,박세혁\n안중열,도태훈\n박민우\n오영수\n서호철\n오태양\n윤형준\n김한별\n김주원,천재환\n김성욱\n손아섭\n박건우\n한석현


In [113]:
positions =['투수','포수','내야수','외야수']
for position in positions:
    df[position] = df[position].apply(lambda x : x.split('\n'))

## 기사 토큰화 환 데이터 셋 준비

토큰화한 데이터 셋을 순회하며 특정 기사에 선수 이름이 포함되어 있을 경우, 함께 언급된 일반 명사, 고유 명사의 빈도수를 세어서 딕셔너리에 추가합니다.

In [114]:
import pandas as pd
from tqdm import tqdm

In [115]:
df = pd.read_csv("./data/baseball_tokenized_10K.csv")

In [116]:
total_players = []

for players_name in df.values.flatten():
    total_players.append(players_name)
total_players = set(total_players)


In [117]:
df.values.flatten()

array(['https://sports.news.naver.com/news?oid=109&aid=0004760729',
       20221221, '"성적 향상에 도움이 된다면..." 원태인, 미국 플로리다로 떠나는 이유', ...,
       "프로야구 SSG가 통합 우승을 이끈 김원형 감독과 3년 총액 22억 원에 재계약했습니다. SSG 구단은 김원형 감독과 계약기간 3년에 계약금 7억 원, 연봉 5억 원 등 첫 번째 재계약 감독 사상 역대 최고인 총액 22억 원에 계약했다고 밝혔습니다. 현역 감독 중에서도 LG 염경엽 감독의 3년 21억 원을 넘어 최고 대우입니다. 2021시즌 처음 사령탑에 오른 김 감독은 올해 KBO리그 최초로 개막부터 마지막까지 1위를 유지하는 '와이어 투 와이어' 우승에 한국시리즈 우승까지 이끌었습니다.",
       "[('SSG', 'NNP'), (',', 'SP'), ('김원형', 'NNP'), ('감독', 'NNG'), ('과', 'JC'), ('3', 'SN'), ('년', 'NNB'), ('22', 'SN'), ('억', 'NR'), ('원', 'NNB'), ('에', 'JKB'), ('재계약', 'NNG'), ('‥', 'SW'), ('현역', 'NNP'), ('최고', 'NNG'), ('대우', 'NNP')]",
       '[(\'프로\', \'NNP\'), (\'야구\', \'NNP\'), (\'SSG\', \'NNP\'), (\'가\', \'JKS\'), (\'통합\', \'NNG\'), (\'우승\', \'NNG\'), (\'을\', \'JKO\'), (\'이끌\', \'VV\'), (\'ㄴ\', \'ETM\'), (\'김원형\', \'NNP\'), (\'감독\', \'NNG\'), (\'과\', \'JC\'), (\'3\', \'SN\'), (\'년\', \'NNB\'), (\'총액\', \'NNG\'), (\'22\', \'SN\'), (\'억\', \'NR\'), (\'원\', \'NNB\'),

### 결측치 확인 및 제거

In [118]:
df.isnull().sum()






url               0
datetime_str      0
title             0
content           0
title_tokens      0
content_tokens    1
dtype: int64

In [119]:
df = df.dropna()



### 명사 추출
제목과 본문을 토큰화한 문자열에서 일반 명사와 고유 명사만 추출하여 데이터 프레임의 컬럼으로 추가합니다.

In [120]:
def extract_nouns(tokens_str):
    tokens = eval(tokens_str)
    nouns = [text for text, tag in tokens if tag in ("NNP", "NNG")]
    return nouns

In [121]:
tqdm.pandas()
df["content_nouns"] = df["content_tokens"].progress_apply(extract_nouns)

100%|█████████████████████████████████████████████████████████████████████████████| 9999/9999 [00:17<00:00, 564.45it/s]


In [122]:
df.iloc[0]['content_tokens']

'[(\'"\', \'SS\'), (\'구속\', \'NNP\'), (\',\', \'SP\'), (\'구위\', \'NNG\'), (\',\', \'SP\'), (\'컨트롤\', \'NNP\'), (\',\', \'SP\'), (\'이닝\', \'NNP\'), (\'소화\', \'NNP\'), (\'능력\', \'NNG\'), (\'등\', \'NNB\'), (\'모든\', \'MM\'), (\'면\', \'NNG\'), (\'에서\', \'JKB\'), (\'현재\', \'MAG\'), (\'능력\', \'NNG\'), (\'보다\', \'JKB\'), (\'한\', \'MM\'), (\'단계\', \'NNG\'), (\'향상\', \'NNG\'), (\'하\', \'XSV\'), (\'는\', \'ETM\'), (\'게\', \'NNG\'), (\'목표\', \'NNG\'), (\'다\', \'NNG\'), (\'"\', \'SS\'), (\'.\', \'SF\'), (\'\\xa0\', \'SW\'), ("\'", \'SS\'), (\'푸른\', \'NNP\'), (\'피\', \'NNG\'), (\'의\', \'JKG\'), (\'에이스\', \'NNP\'), ("\'", \'SS\'), (\'원태인\', \'NNP\'), (\'(\', \'SS\'), (\'22\', \'SN\'), (\'·\', \'SP\'), (\'삼성\', \'NNP\'), (\')\', \'SS\'), (\'이\', \'MM\'), (\'성적\', \'NNG\'), (\'향상\', \'NNG\'), (\'을\', \'JKO\'), (\'위하\', \'VV\'), (\'아\', \'EC\'), (\'통\', \'MAG\'), (\'크\', \'VA\'), (\'ㄴ\', \'ETM\'), (\'투자\', \'NNG\'), (\'에\', \'JKB\'), (\'나서\', \'VV\'), (\'ㄴ다\', \'EF\'), (\'.\', \'SF\'), (\'원태인\', \'NNP\')

In [123]:
df

,url,datetime_str,title,content,title_tokens,content_tokens,content_nouns
0,https://sports.news.naver.com/news?oid=109&aid...,20221221,"""성적 향상에 도움이 된다면..."" 원태인, 미국 플로리다로 떠나는 이유","""구속, 구위, 컨트롤, 이닝 소화 능력 등 모든 면에서 현재 능력보다 한 단계 향...","[('""', 'SS'), ('성적', 'NNP'), ('향상', 'NNG'), ('...","[('""', 'SS'), ('구속', 'NNP'), (',', 'SP'), ('구위...","[구속, 구위, 컨트롤, 이닝, 소화, 능력, 면, 능력, 단계, 향상, 게, 목표..."
1,https://sports.news.naver.com/news?oid=421&aid...,20220325,"""재환이와 승부를 안한다""…김태형 감독의 라인업 고민, '키'는 페르난데스",(서울=뉴스1) 서장원 기자 = 김태형 두산 베어스 감독이 외국인 타자 호세 페르난...,"[('""', 'SS'), ('재화', 'NNP'), ('ㄴ', 'ETM'), ('이...","[('(', 'SS'), ('서울', 'NNP'), ('=', 'SW'), ('뉴스...","[서울, 뉴스1, 서장원, 기자, 김태형, 두산, 베어스, 감독, 외국인, 타자, ..."
2,https://sports.news.naver.com/news?oid=108&aid...,20220401,"두산, 2022시즌 팬북 발행... 엠블럼 스티커-선수단 포스터 제공","두산 베어스가 2022시즌 개막을 앞두고 새로운 팬북을 내놓았다.두산은 1일 ""20...","[('두산', 'NNP'), (',', 'SP'), ('2022', 'SN'), (...","[('두산', 'NNP'), ('베어스', 'NNP'), ('가', 'JKS'), ...","[두산, 베어스, 시즌, 개막, 팬, 북, 두산, 2022년, 팬, 북, 발행, 구..."
3,https://sports.news.naver.com/news?oid=529&aid...,20220125,"'캡틴 어게인' 박경수 ""후배들아, 우리 수원에서 우승 한 번만 더 하자."" [SP...","-'2021 KS MVP' KT WIZ 내야수 박경수, 2022년 주장으로 다시 돌...","[(""'"", 'SS'), ('캡틴', 'NNP'), ('어게인', 'NNP'), (...","[('-', 'SS'), (""'"", 'SS'), ('2021', 'SN'), ('K...","[KT, 내야수, 박경수, 주장, 후배, 숟가락, 팬, 뒤, 목발, 세리, 모니, ..."
4,https://sports.news.naver.com/news?oid=477&aid...,20220426,"""포지션 변경 고려""…NC 정현, 자취 감춘 이유 있었네","""포지션 변경도 고려하고 있다.""이동욱 NC 다이노스 감독이 26일 잠실야구장에서 ...","[('""', 'SS'), ('포지션', 'NNP'), ('변경', 'NNG'), (...","[('""', 'SS'), ('포지션', 'NNP'), ('변경', 'NNG'), (...","[포지션, 변경, 고려, 이동욱, NC, 다이노스, 감독, 잠실야구장, 두산, 베어..."
...,...,...,...,...,...,...,...
9995,https://sports.news.naver.com/news?oid=109&aid...,20220525,"양현종 타이거즈 최다승 새 역사…KIA, 대구 3연전 위닝 시리즈 예약 [대구 리뷰]",KIA가 25일 대구 원정 경기에서 삼성을 11-5로 꺾고 위닝 시리즈를 예약했다....,"[('양현종', 'NNP'), ('타이', 'NNP'), ('거즈', 'NNP'),...","[('KIA', 'NNP'), ('가', 'JKS'), ('25', 'SN'), (...","[KIA, 대구, 원정, 경기, 삼성, -5, 시리즈, 예약, 에이스, 양현종, 타..."
9996,https://sports.news.naver.com/news?oid=117&aid...,20221105,"'3회 5점 빅이닝' 키움, SSG에 6-3 승리…시리즈 2승 2패 균형 [KS]",3회말 빅이닝이 승부를 갈랐다. 키움이 웃었다.키움 히어로즈는 5일 서울 고척스카이...,"[(""'"", 'SS'), ('3', 'SN'), ('회', 'NNB'), ('5',...","[('3', 'SN'), ('회', 'NNB'), ('말', 'NNB'), ('빅'...","[빅, 이닝, 승부, 키움, 키움, 히어로즈, 서울, 고척스카이돔, 신한은행, KB..."
9997,https://sports.news.naver.com/news?oid=031&aid...,20220922,"LG, 은퇴투어 롯데 이대호에 목각 기념패·메시지 보드 전달",롯데 자이언츠 이대호(내야수)가 LG 트윈스를 상대로 올 시즌 원정팀 대상 은퇴투어...,"[('LG', 'NNP'), (',', 'SP'), ('은퇴', 'NNG'), ('...","[('롯데', 'NNP'), ('자이언츠', 'NNP'), ('이대호', 'NNP'...","[롯데, 자이언츠, 이대호, 내야수, LG, 트윈스, 상대로, 시즌, 원정, 팀, ..."
9998,https://sports.news.naver.com/news?oid=410&aid...,20220630,“안우진은 분명 한 단계 성장했다” [MK현장],“(안)우진이는 분명 한 단계 성장했습니다.”지금의 키움 히어로즈를 이야기할 때 안...,"[('“', 'SS'), ('안우진', 'NNP'), ('은', 'JX'), ('분...","[('“', 'SS'), ('(', 'SS'), ('안', 'NNP'), (')',...","[안, 우진, 단계, 성장, 지금, 키움, 히어로즈, 이야기, 때, 안우진, 현시점..."


### 동시 출현 빈도 집계
특정 선수의 이름과 함께 등장한 명사를 defaultdict와 Counter를 이용해서 집계합니다.

In [124]:
from collections import Counter, defaultdict


word_count_dict = defaultdict(Counter)

In [125]:
def count_words(content_nouns):
    content_nouns_set = set(content_nouns)
    player_name_nouns= total_players.intersection(content_nouns_set)
    
    for player_name in player_name_nouns:
        c = Counter([x for x in content_nouns if x != player_name])
        word_count_dict[player_name] += c
    
    


In [131]:
from tqdm import tqdm
content_nouns_series = df["content_nouns"]
for content_nouns in tqdm(df["content_nouns"]):
    count_words(content_nouns)

100%|███████████████████████████████████████████████████████████████████████████| 9999/9999 [00:00<00:00, 18568.65it/s]


In [132]:
word_count_dict["김광현"].most_common(10)

[]

## word cloud 시각화

함께 등장하는 단어를 세었으니 이제 워드 클라우드를 만들어보겠습니다. 폰트 적용 부분은 아래 가이드를 읽어보고, 본인의 환경에 맞게 코드를 수정하면 됩니다.

- [윈도우 폰트 적용 가이드](https://doitgrow.com/34#:~:text=%EC%9B%8C%EB%93%9C%ED%81%B4%EB%9D%BC%EC%9A%B0%EB%93%9C%EC%97%90%20%EC%82%AC%EC%9A%A9%ED%95%A0%20%ED%8F%B0%ED%8A%B8,%EA%B2%B0%EA%B3%BC%EB%A5%BC%20%ED%99%95%EC%9D%B8%ED%95%A0%20%EC%88%98%20%EC%9E%88%EC%8A%B5%EB%8B%88%EB%8B%A4.)
- [맥 폰트 적용 가이드](https://business-analytics.tistory.com/3)

In [128]:
from wordcloud import WordCloud
from matplotlib import pyplot as plt

def visualize_wordcloud(word_count, color):
    wordcloud = WordCloud(
        font_path="/Library/Fonts/NanumGothic.otf",
        width=1000, 
        height=400, 
        scale=2.0, 
        background_color='white', 
        colormap=color,
        max_font_size=150
    ).generate_from_frequencies(word_count)
    plt.figure(figsize=(10, 10))
    plt.imshow(wordcloud)

In [129]:
visualize_wordcloud(word_count_dict["김광현"], "Reds")

ValueError: We need at least 1 word to plot a word cloud, got 0.

## 정리
이번 챕터에서는 자연어 데이터 시각화 하면 가장 먼저 떠오르는 워드 클라우드를 만들어 보았습니다. 사실 워드 클라우드는 직관적으로 의미를 전달하는 것이 어려워서 잘 사용되지 않습니다. 이어지는 챕터들에서 더 정교화 된 자연어처리 알고리즘들을 배워보겠습니다.